In [12]:
import h5py
import cupy as cp
import matplotlib.pyplot as plt
import  pandas as pd
import  os
import numpy as np
from sklearn.preprocessing import MinMaxScaler


In [ ]:
#查看cuda版本
import torch
print(torch.version.cuda)

11.3


In [13]:
#读取数据
f = h5py.File('/content/drive/MyDrive/ml/dataset_convlstm/data_git_version.h5', 'r')
data_o = f['data']
data_o = cp.array(data_o)
print(data_o.shape)
#数据变成四维(1488, 10000, 3)->(1488,100,100,3)
data = cp.zeros((1488,100,100,1))
for i in range(1488):
    data[i] = data_o[i,:,1].reshape(100,100,1)
print(data.shape)



(1488, 10000, 3)
(1488, 100, 100, 1)


In [11]:
#查看最大值
print(np.max(data))

4434.6990000000005


In [14]:
#归一化 除最大值
data=data/4435
'''data=data.reshape(14880000,1)
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data.get())
data=data.reshape(1488, 100, 100, 1)'''


'data=data.reshape(14880000,1)\nscaler = MinMaxScaler(feature_range=(0, 1))\ndata = scaler.fit_transform(data.get())\ndata=data.reshape(1488, 100, 100, 1)'

In [15]:
#拆分数据集
train_size = int(1488* 0.8)
trainlist=data[:train_size]
testlist=data[train_size:]
print(trainlist.shape)
print(testlist.shape)

(1190, 100, 100, 1)
(298, 100, 100, 1)


In [17]:

#查看trainlist第一个维度的大小
print(trainlist.shape[0])

1190


In [19]:
#滑窗
def create_dataset(dataset, look_back):
#这里的look_back与timestep相同
    dataX, dataY = [], []
    for i in range(dataset.shape[0]-look_back-1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return cp.array(dataX),cp.array(dataY)
look_back = 24
trainX,trainY  = create_dataset(trainlist,look_back)
testX,testY = create_dataset(testlist,look_back)
trainX = cp.reshape(trainX,(trainX.shape[0],trainX.shape[1],trainX.shape[2],trainX.shape[3],1))
testX = cp.reshape(testX,(testX.shape[0],testX.shape[1],testX.shape[2],testX.shape[3],1))
trainY = cp.reshape(trainY,(trainY.shape[0],trainY.shape[1],trainY.shape[2],1))
testY = cp.reshape(testY,(testY.shape[0],testY.shape[1],testY.shape[2],1))
print(trainX.shape)
print(trainY.shape)

(1165, 24, 100, 100, 1)
(1165, 100, 100, 1)


In [20]:
#保存为h5文件
f1= h5py.File('/content/drive/Shareddrives/gsd/trainX1.h5', 'w')
f1.create_dataset('trainX', data=trainX.get())
f1.close()
f1 = h5py.File('/content/drive/Shareddrives/gsd/trainY1.h5', 'w')
f1.create_dataset('trainY', data=trainY.get())
f1.close()
f1 = h5py.File('/content/drive/Shareddrives/gsd/testX1.h5', 'w')
f1.create_dataset('testX', data=testX.get())
f1.close()
f1 = h5py.File('/content/drive/Shareddrives/gsd/testY1.h5', 'w')
f1.create_dataset('testY', data=testY.get())
f1.close()